# Reading the HTML of Rotten Tomatoes


First let's make sure we are pulling the link to where we will be takin information

In [1]:
import requests
from bs4 import BeautifulSoup

r = requests.get('https://www.rottentomatoes.com/browse/movies_at_home/sort:popular?page=1')
soup = BeautifulSoup(r.content, 'lxml')

a_href=soup.find_all()

test=soup.find_all('div', {"class":"discovery-tiles__wrap"})

links = []

for a in test:
    all_a = a.find_all('a')
    for link in all_a:
        links.append(link.get('href'))
        


links_test=[]     
for link in links:
    link=f'https://www.rottentomatoes.com{link}'
    links_test.append(link)
        
succesful_links = links_test

In [2]:
a=links.copy()
names_of_movie=[]
for item in a:
    names_of_movie.append(item.replace('/m/',''))
    
names_of_movie[0]

    
    

'doctor_strange_in_the_multiverse_of_madness'

Now that we have synthesized a redirectory in the form of a list have the freedom to call the object from the list

In [3]:
succesful_links

['https://www.rottentomatoes.com/m/doctor_strange_in_the_multiverse_of_madness',
 'https://www.rottentomatoes.com/m/everything_everywhere_all_at_once',
 'https://www.rottentomatoes.com/m/the_sea_beast',
 'https://www.rottentomatoes.com/m/thor_ragnarok_2017',
 'https://www.rottentomatoes.com/m/thor',
 'https://www.rottentomatoes.com/m/hustle_2022',
 'https://www.rottentomatoes.com/m/happening',
 'https://www.rottentomatoes.com/m/girl_in_the_picture_2022',
 'https://www.rottentomatoes.com/m/x_2022',
 'https://www.rottentomatoes.com/m/the_innocents_2022',
 'https://www.rottentomatoes.com/m/spiderhead',
 'https://www.rottentomatoes.com/m/neptune_frost',
 'https://www.rottentomatoes.com/m/the_man_from_toronto_2022',
 'https://www.rottentomatoes.com/m/leave_no_trace',
 'https://www.rottentomatoes.com/m/beavis_and_butt_head_do_the_universe',
 'https://www.rottentomatoes.com/m/the_batman',
 'https://www.rottentomatoes.com/m/last_night_in_soho',
 'https://www.rottentomatoes.com/m/moon_66_questi

We will have to visit each one of these urls inorder to extract information for our data frame

Let's test it on a single scrape

In [6]:
test_dict = {}

for i in range(0,len(succesful_links)):
    
    r = requests.get(succesful_links[i])    
    soup = BeautifulSoup(r.text, "html.parser")
    audience_score= soup.find("score-board").get('audiencescore')
    tomato_score=soup.find("score-board").get('tomatometerscore')
    rating =soup.find("score-board").get('rating')
    
    test_dict[names_of_movie[i]]={"audience_score":audience_score,"tomato_score":tomato_score,"rating":rating}
    
print(test_dict)


    
    

{'doctor_strange_in_the_multiverse_of_madness': {'audience_score': '85', 'tomato_score': '74', 'rating': 'PG-13'}, 'everything_everywhere_all_at_once': {'audience_score': '89', 'tomato_score': '95', 'rating': 'R'}, 'the_sea_beast': {'audience_score': '89', 'tomato_score': '94', 'rating': 'PG'}, 'thor_ragnarok_2017': {'audience_score': '87', 'tomato_score': '93', 'rating': 'PG-13'}, 'thor': {'audience_score': '76', 'tomato_score': '77', 'rating': 'PG-13'}, 'hustle_2022': {'audience_score': '93', 'tomato_score': '92', 'rating': 'R'}, 'happening': {'audience_score': '79', 'tomato_score': '99', 'rating': 'R'}, 'girl_in_the_picture_2022': {'audience_score': '87', 'tomato_score': '94', 'rating': ''}, 'x_2022': {'audience_score': '75', 'tomato_score': '95', 'rating': 'R'}, 'the_innocents_2022': {'audience_score': '70', 'tomato_score': '97', 'rating': ''}, 'spiderhead': {'audience_score': '30', 'tomato_score': '40', 'rating': 'R'}, 'neptune_frost': {'audience_score': '63', 'tomato_score': '97'

Great Now we have a way of generating a dictionary with scraped items. However I want more information.

In [7]:
r = requests.get(succesful_links[1])    
soup = BeautifulSoup(r.text, "html.parser")
movie_info_keys=soup.find("ul",{"class":"content-meta info"}).find_all("li")
movie_info_values=soup.find("ul",{"class":"content-meta info"}).find_all("li")

###keys 
m_i_k_list = []
for li in movie_info_keys:
    li = li.find('div').text
    li=li.replace(":","")
    m_i_k_list.append(li)
###values 
m_i_v_list = []
for meta in movie_info_values:
    meta=meta.find('div', {"class":"meta-value"}).text
    meta=meta.replace(" ","")
    meta=meta.replace("\n","")
    m_i_v_list.append(meta)
    

m_i_k_list,m_i_v_list



(['Rating',
  'Genre',
  'Original Language',
  'Director',
  'Producer',
  'Writer',
  'Release Date (Theaters)',
  'Release Date (Streaming)',
  'Box Office (Gross USA)',
  'Runtime',
  'Distributor',
  'Sound Mix',
  'Aspect Ratio'],
 ['R(SexualMaterial|Language|SomeViolence)',
  'Comedy,Adventure,Fantasy',
  'English',
  'DanKwan,DanielScheinert',
  'JoeRusso,AnthonyRusso,MikeLarocca,DanKwan,DanielScheinert,JonathanWang',
  'DanKwan,DanielScheinert',
  'Apr8,2022\xa0wide',
  'Jun7,2022',
  '$67.6M',
  '2h12m',
  'A24',
  'DolbyDigital',
  'Flat(1.85:1)'])

I want to create a forloop that can do this for each entry and also append to our test dictionary. Let's see if I can fuse the code above and debug from there. 

In [8]:
tomato_dict = {}
tomato_dict_more_info = {}
for i in range(0,len(succesful_links)):
    
    r = requests.get(succesful_links[i])    
    soup = BeautifulSoup(r.text, "html.parser")
    audience_score= soup.find("score-board").get('audiencescore')
    tomato_score=soup.find("score-board").get('tomatometerscore')
    rating =soup.find("score-board").get('rating')
    movie_info_keys=soup.find("ul",{"class":"content-meta info"}).find_all("li")
    movie_info_values=soup.find("ul",{"class":"content-meta info"}).find_all("li")
    
    tomato_dict[names_of_movie[i]]={"audience_score":audience_score,"tomato_score":tomato_score,"rating":rating}
###keys 
    m_i_k_list = []
    for li in movie_info_keys:
        li = li.find('div').text
        li=li.replace(":","")
        m_i_k_list.append(li)
###values 
    m_i_v_list = []
    for meta in movie_info_values:
        meta=meta.find('div', {"class":"meta-value"}).text
        meta=meta.replace(" ","")
        meta=meta.replace("\n","")
        m_i_v_list.append(meta)
        
    
    zip_iterator = zip(m_i_k_list,m_i_v_list)
    tomato_dict_more_info = {names_of_movie[i]:dict(zip_iterator)}
           
    
    
    print(tomato_dict_more_info)
                                  
       

    
    
                                            
    



{'doctor_strange_in_the_multiverse_of_madness': {'Rating': 'PG-13(FrighteningImages|Action|IntenseSequencesofViolence|SomeLanguage)', 'Genre': 'Action,Adventure,Fantasy', 'Original Language': 'English', 'Director': 'SamRaimi', 'Producer': 'KevinFeige', 'Writer': 'MichaelWaldron', 'Release Date (Theaters)': 'May6,2022\xa0wide', 'Release Date (Streaming)': 'Jun22,2022', 'Box Office (Gross USA)': '$411.0M', 'Runtime': '2h6m', 'Distributor': 'WaltDisneyPictures', 'Sound Mix': 'DolbyAtmos,DolbyDigital,DTS', 'Aspect Ratio': 'Scope(2.35:1)', 'View the collection': 'MarvelCinematicUniverse'}}
{'everything_everywhere_all_at_once': {'Rating': 'R(SexualMaterial|Language|SomeViolence)', 'Genre': 'Comedy,Adventure,Fantasy', 'Original Language': 'English', 'Director': 'DanKwan,DanielScheinert', 'Producer': 'JoeRusso,AnthonyRusso,MikeLarocca,DanKwan,DanielScheinert,JonathanWang', 'Writer': 'DanKwan,DanielScheinert', 'Release Date (Theaters)': 'Apr8,2022\xa0wide', 'Release Date (Streaming)': 'Jun7,202

{'eternals': {'Rating': 'PG-13(FantasyViolenceandAction|BriefSexuality|SomeLanguage)', 'Genre': 'Action,Adventure,Fantasy', 'Original Language': 'English', 'Director': 'ChloéZhao', 'Producer': 'KevinFeige,NateMoore', 'Writer': 'ChloéZhao,RyanFirpo,KazFirpo', 'Release Date (Theaters)': 'Nov5,2021\xa0wide', 'Release Date (Streaming)': 'Jan12,2022', 'Box Office (Gross USA)': '$164.9M', 'Runtime': '2h37m', 'Distributor': 'WaltDisney', 'Sound Mix': 'DolbyAtmos,DolbyDigital', 'Aspect Ratio': 'Scope(2.35:1)', 'View the collection': 'MarvelCinematicUniverse'}}
{'thor_the_dark_world': {'Rating': 'PG-13(SomeSuggestiveContent|IntenseSci-FiAction/Violence)', 'Genre': 'Action,Adventure,Fantasy', 'Original Language': 'English', 'Director': 'AlanTaylor', 'Producer': 'KevinFeige', 'Writer': 'ChristopherYost,ChristopherMarkus,StephenMcFeely', 'Release Date (Theaters)': 'Nov8,2013\xa0wide', 'Release Date (Streaming)': 'Feb25,2014', 'Box Office (Gross USA)': '$206.4M', 'Runtime': '1h51m', 'Distributor': 

Now I can create two dictionaries for every link. However, the link is currently limited to whatever is displayed on 
the page. There is a button that has a link which once interacted with will enlongate the list of information I am 
scraping. Both dictionaries share the same Key Values which will help when merging them into one data frame

The code below opens my web page and successfully extends it, however it only does this for the get('initial page') when I have to repeat this command until it cannot execute anymore

i will set count = 1 (first page) and have the link that is scraped update everytime loud_botton.click() is executed and merge all of my 
previous work 

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


    


count=1    
url=f'https://www.rottentomatoes.com/browse/movies_at_home/sort:popular?page={count}'
driver = webdriver.Chrome(executable_path='./chromedriver')
driver.get=url
time.sleep(1)
load_button = driver.find_element(By.XPATH,'//*[@id="main-page-content"]/div/div[5]/button')
load_button.click()
count+=1
time.sleep(1)
# if count>50:
    

        
#         r = requests.get(url)
#         soup = BeautifulSoup(r.content, 'lxml')

#         a_href=soup.find_all()

#         test=soup.find_all('div', {"class":"discovery-tiles__wrap"})

#         links = []

#         for a in test:
#             all_a = a.find_all('a')
#             for link in all_a:
#                 links.append(link.get('href'))


#         links_test=[]     
#         for link in links:
#             link=f'https://www.rottentomatoes.com{link}'
#             links_test.append(link)

#         succesful_links = links_test

#         tomato_dict = {}
#         tomato_dict_more_info = {}

#         for i in range(0,len(succesful_links)):

#             if count>500:

#                 r = requests.get(succesful_links[i])    
#                 soup = BeautifulSoup(r.text, "html.parser")
#                 audience_score= soup.find("score-board").get('audiencescore')
#                 tomato_score=soup.find("score-board").get('tomatometerscore')
#                 rating =soup.find("score-board").get('rating')
#                 movie_info_keys=soup.find("ul",{"class":"content-meta info"}).find_all("li")
#                 movie_info_values=soup.find("ul",{"class":"content-meta info"}).find_all("li")

#                 tomato_dict[names_of_movie[i]]={"audience_score":audience_score,"tomato_score":tomato_score,"rating":rating}
#         ###keys 
#                 m_i_k_list = []
#                 for li in movie_info_keys:
#                     li = li.find('div').text
#                     li=li.replace(":","")
#                     m_i_k_list.append(li)
#         ###values 
#                 m_i_v_list = []
#                 for meta in movie_info_values:
#                     meta=meta.find('div', {"class":"meta-value"}).text
#                     meta=meta.replace(" ","")
#                     meta=meta.replace("\n","")
#                     m_i_v_list.append(meta)


#                 zip_iterator = zip(m_i_k_list,m_i_v_list)
#                 tomato_dict_more_info = {names_of_movie[i]:dict(zip_iterator)}



#                 print(tomato_dict_more_info)

       


/var/folders/xp/ksk4jb6j25df8dk_14f29cqw0000gn/T/ipykernel_8998/3347784395.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-page-content"]/div/div[5]/button"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x0000000108b27079 chromedriver + 4444281
1   chromedriver                        0x0000000108ab3403 chromedriver + 3970051
2   chromedriver                        0x000000010874e038 chromedriver + 409656
3   chromedriver                        0x00000001087842fd chromedriver + 631549
4   chromedriver                        0x0000000108784581 chromedriver + 632193
5   chromedriver                        0x00000001087b64f4 chromedriver + 836852
6   chromedriver                        0x00000001087a17bd chromedriver + 751549
7   chromedriver                        0x00000001087b4221 chromedriver + 827937
8   chromedriver                        0x00000001087a1683 chromedriver + 751235
9   chromedriver                        0x0000000108777a45 chr

In [ ]:
r = requests.get('https://www.rottentomatoes.com/browse/movies_at_home/sort:popular?page=340')
soup = BeautifulSoup(r.content, 'lxml')

a_href=soup.find_all()

test=soup.find_all('div', {"class":"discovery-tiles__wrap"})

links = []

for a in test:
    all_a = a.find_all('a')
    for link in all_a:
        links.append(link.get('href'))

links_test=[]     
for link in links:
    link=f'https://www.rottentomatoes.com{link}'
    links_test.append(link)

succesful_links = links_test
    
tomato_dict = {}
tomato_dict_more_info = {}
    
for i in range(0,len(succesful_links)):

       

    r = requests.get(succesful_links[i])    
    soup = BeautifulSoup(r.text, "html.parser")
    audience_score= soup.find("score-board").get('audiencescore')
    tomato_score=soup.find("score-board").get('tomatometerscore')
    rating =soup.find("score-board").get('rating')
    movie_info_keys=soup.find("ul",{"class":"content-meta info"}).find_all("li")
    movie_info_values=soup.find("ul",{"class":"content-meta info"}).find_all("li")

#     tomato_dict[names_of_movie[i]]={"audience_score":audience_score,"tomato_score":tomato_score,"rating":rating}
    ###keys 
    m_i_k_list = []
    for li in movie_info_keys:
        li = li.find('div').text
        li=li.replace(":","")
        m_i_k_list.append(li)
    ###values 
    m_i_v_list = []
    for meta in movie_info_values:
        meta=meta.find('div', {"class":"meta-value"}).text
        meta=meta.replace(" ","")
        meta=meta.replace("\n","")
        m_i_v_list.append(meta)


    zip_iterator = zip(m_i_k_list,m_i_v_list)
    tomato_dict_more_info = {names_of_movie[i]:dict(zip_iterator)}



    print(tomato_dict_more_info)

        
    